In [2]:
import os
import shutil

import gdown
import numpy as np
import pandas as pd

# Download files from Google Drive
# Based on data from: http://insideairbnb.com/get-the-data/
file_id_1 = "1m185vTdh-u7_A2ZElBvUD4SCO6oETll2"
file_id_2 = "1w41V1oWHJrBdaNJJQ4oxVBuml5CO7MQX"
downloaded_file_1 = "listings_project.pkl"
downloaded_file_2 = "calendar_project.parquet"
# Download the files from Google Drive
gdown.download(id=file_id_1, output=downloaded_file_1)
gdown.download(id=file_id_2, output=downloaded_file_2)

Downloading...
From: https://drive.google.com/uc?id=1m185vTdh-u7_A2ZElBvUD4SCO6oETll2
To: C:\Users\zisto\listings_project.pkl
100%|██████████| 1.42M/1.42M [00:00<00:00, 17.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1w41V1oWHJrBdaNJJQ4oxVBuml5CO7MQX
To: C:\Users\zisto\calendar_project.parquet
100%|██████████| 1.23M/1.23M [00:00<00:00, 13.9MB/s]


'calendar_project.parquet'

In [3]:
# Show all columns (instead of cascading columns in the middle)
pd.set_option("display.max_columns", None)
# Don't show numbers in scientific notation
pd.set_option("display.float_format", "{:.2f}".format)

In [4]:
df_list = pd.read_pickle("listings_project.pkl")
df_cal = pd.read_parquet("calendar_project.parquet", engine="pyarrow")

In [5]:
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6165 entries, 0 to 6172
Data columns (total 34 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    6165 non-null   int64  
 1   host_acceptance_rate                  5365 non-null   float64
 2   host_is_superhost                     6165 non-null   object 
 3   host_listings_count                   6165 non-null   int64  
 4   host_total_listings_count             6165 non-null   int64  
 5   neighbourhood_cleansed                6165 non-null   object 
 6   latitude                              6165 non-null   float64
 7   longitude                             6165 non-null   float64
 8   room_type                             6165 non-null   object 
 9   accommodates                          6165 non-null   int64  
 10  bedrooms                              5859 non-null   float64
 11  beds             

In [6]:
df_list.discount_per_5_days_booked.head(5)

0    5%
1    5%
2    7%
3    6%
4    9%
Name: discount_per_5_days_booked, dtype: object

In [7]:
df_list["discount_per_5_days_booked"] = (
    df_list["discount_per_5_days_booked"]
    .str.replace("%", "", regex=True)
    .astype("float")
    * 0.01
)
df_list["discount_per_10_days_booked"] = (
    df_list["discount_per_10_days_booked"]
    .str.replace("%", "", regex=True)
    .astype("float")
    * 0.01
)
df_list["discount_per_30_and_more_days_booked"] = (
    df_list["discount_per_30_and_more_days_booked"]
    .str.replace("%", "", regex=True)
    .astype("float")
    * 0.01
)

In [8]:
df_list[["host_is_superhost", "instant_bookable", "has_availability"]].head(5)

,host_is_superhost,instant_bookable,has_availability
0,f,t,t
1,t,f,t
2,f,f,t
3,f,f,t
4,t,f,t


In [9]:
df_list["host_is_superhost"] = (
    df_list["host_is_superhost"].replace({"f": False, "t": True}).astype("bool")
)
df_list["instant_bookable"] = (
    df_list["instant_bookable"].replace({"f": False, "t": True}).astype("bool")
)
df_list["has_availability"] = (
    df_list["has_availability"].replace({"f": False, "t": True}).astype("bool")
)

In [10]:
df_list[["price", "price_per_person", "minimum_price", 'service_cost']].head(5)

,price,price_per_person,minimum_price,service_cost
0,$88.00,$44,$176,$4.99
1,$105.00,$52.5,$315,$4.99
2,$152.00,$38,$304,$4.99
3,$87.00,$43.5,$174,$4.99
4,$160.00,$40,$320,$4.99


In [11]:
df_list["price"] = (
    df_list["price"]
    .str.replace("$", "", regex=True)
    .str.replace(",", "", regex=True)
    .astype("float")
)
df_list["price_per_person"] = (
    df_list["price_per_person"]
    .str.replace("$", "", regex=True)
    .str.replace(",", "", regex=True)
    .astype("float")
)
df_list["minimum_price"] = (
    df_list["minimum_price"]
    .str.replace("$", "", regex=True)
    .str.replace(",", "", regex=True)
    .astype("float")
)
df_list["service_cost"] = (
    df_list["service_cost"]
    .str.replace("$", "", regex=True)
    .str.replace(",", "", regex=True)
    .astype("float")
)

In [12]:
df_list = df_list.rename(columns={"price": "price_in_dollar", "neighbourhood_cleansed": "neighbourhood"})

In [13]:
df_list = df_list.astype(
    {
        "neighbourhood": "category",
        "room_type": "category",
    }
)

In [14]:
df_list = df_list.drop(
    columns=[
        "host_listings_count",
        "host_total_listings_count",
        "availability_60",
        "availability_90",
        "availability_365",
        "number_of_reviews",
        "number_of_reviews_ltm",
        "reviews_per_month",
    ]
)

In [15]:
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6165 entries, 0 to 6172
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    6165 non-null   int64   
 1   host_acceptance_rate                  5365 non-null   float64 
 2   host_is_superhost                     6165 non-null   bool    
 3   neighbourhood                         6165 non-null   category
 4   latitude                              6165 non-null   float64 
 5   longitude                             6165 non-null   float64 
 6   room_type                             6165 non-null   category
 7   accommodates                          6165 non-null   int64   
 8   bedrooms                              5859 non-null   float64 
 9   beds                                  6082 non-null   float64 
 10  amenities                             6165 non-null   int64   
 11  pric

In [18]:
df_list["room_type"].unique()

['Private room', 'Entire home/apt', 'Hotel room', 'Shared room']
Categories (4, object): ['Entire home/apt', 'Hotel room', 'Private room', 'Shared room']

In [19]:
def fill_empty_bedrooms(accommodates: int, bedrooms: int, room_type: str) -> int:
    if (room_type == "Private room") or (room_type == "Shared room"):
        return 1
    elif (room_type == "Hotel room") or (room_type == "Entire home/apt"):
        return np.ceil(accommodates / 2)
    else:
        return bedrooms

In [20]:
%%timeit -r 4 -n 100

temp_df = df_list.copy()  # Deep copy of the df, not a "view"
temp_df["rooms"] = df_list[["accommodates", "bedrooms", "room_type"]].apply(
    lambda x: fill_empty_bedrooms(x["accommodates"], x["bedrooms"], x["room_type"]),
    axis=1,
)

91.3 ms ± 1.35 ms per loop (mean ± std. dev. of 4 runs, 100 loops each)


In [22]:
df_list["bedrooms"] = df_list[["accommodates", "bedrooms", "room_type"]].apply(
    lambda x: fill_empty_bedrooms(x["accommodates"], x["bedrooms"], x["room_type"]),
    axis=1,
)

In [23]:
%%timeit -r 4 -n 100

temp_df = df_list.copy()


temp_df["beds"] = temp_df.bedrooms
priv_shared_mask = (temp_df.room_type == "Private room") | (
    temp_df.room_type == "Shared room"
)
temp_df.loc[priv_shared_mask, "beds"] = 1
hotel_apt_mask = (temp_df.room_type == "Hotel room") | (
    temp_df.room_type == "Entire home/apt"
)
temp_df.loc[hotel_apt_mask, "beds"] = np.ceil(temp_df.accommodates / 2)

3.02 ms ± 212 µs per loop (mean ± std. dev. of 4 runs, 100 loops each)


In [25]:
df_list = df_list.dropna(subset=["bedrooms", "beds"])

In [26]:
df_list["beds"] = df_list["beds"].astype("int")
df_list["bedrooms"] = df_list["bedrooms"].astype("int")

In [27]:
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6082 entries, 0 to 6172
Data columns (total 25 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    6082 non-null   int64   
 1   host_acceptance_rate                  5283 non-null   float64 
 2   host_is_superhost                     6082 non-null   bool    
 3   neighbourhood                         6082 non-null   category
 4   latitude                              6082 non-null   float64 
 5   longitude                             6082 non-null   float64 
 6   room_type                             6082 non-null   category
 7   accommodates                          6082 non-null   int64   
 8   bedrooms                              6082 non-null   int32   
 9   beds                                  6082 non-null   int32   
 10  amenities                             6082 non-null   int64   
 11  pric

In [28]:
df_list.head(3)

,id,host_acceptance_rate,host_is_superhost,neighbourhood,latitude,longitude,room_type,accommodates,bedrooms,beds,amenities,price_in_dollar,minimum_nights,maximum_nights,has_availability,availability_30,number_of_reviews_l30d,review_scores_rating,instant_bookable,price_per_person,minimum_price,discount_per_5_days_booked,discount_per_10_days_booked,discount_per_30_and_more_days_booked,service_cost
0,23726706,0.95,False,IJburg - Zeeburgereiland,52.35,4.98,Private room,2,1,1,6,88.00,2,14,True,0,3,4.99,True,44.00,176.00,0.05,0.11,0.16,4.99
1,35815036,1.00,True,Noord-Oost,52.42,4.96,Entire home/apt,2,1,1,5,105.00,3,100,True,4,6,4.96,False,52.50,315.00,0.05,0.12,0.16,4.99
2,31553121,1.00,False,Noord-West,52.43,4.92,Entire home/apt,4,2,3,3,152.00,2,60,True,0,1,4.74,False,38.00,304.00,0.07,0.11,0.22,4.99


In [29]:
# The Calendar DataFrame!
df_cal.head(3)

,listing_id,date,available,price_in_dollar,minimum_nights,maximum_nights
0,23726706,2022-06-05,False,90.00,2,1125
1,23726706,2022-06-06,False,90.00,2,1125
2,23726706,2022-06-07,False,90.00,2,1125


In [30]:
calendar_newdf = df_cal.copy()

include_list = (
    calendar_newdf["minimum_nights"] >= 3
)

In [31]:
calendar_newdf = calendar_newdf.loc[include_list]

In [32]:
calendar_newdf["five_day_dollar_price"] = calendar_newdf["price_in_dollar"] * 5

In [33]:
calendar_summarizeddf = pd.pivot_table(
    data=calendar_newdf,
    index=["listing_id"],
    values=["available", "five_day_dollar_price"],
    aggfunc=np.mean,  # The default aggregation function used
    # for merging multiple related rows of data.
)

calendar_summarizeddf.head(3)

,available,five_day_dollar_price
listing_id,,
2818,0.21,346.90
44391,0.00,1200.00
49552,0.46,1162.50


In [35]:
temp_sum_df = pd.pivot_table(
    data=calendar_newdf,
    index=["listing_id"],
    values=["price_in_dollar"],
    aggfunc=np.max,  # The default aggregation function used
    # for merging multiple related rows of data.
)

In [36]:
final_df = pd.merge(
    df_list,
    calendar_summarizeddf,
    left_on=["id"],
    right_on=["listing_id"],
    how="inner",
)

In [37]:
final_df.groupby(by=["room_type"])[
    [
        "review_scores_rating",
        "five_day_dollar_price",
    ]
].median()

,review_scores_rating,five_day_dollar_price
room_type,,
Entire home/apt,4.88,956.91
Hotel room,4.56,1139.81
Private room,4.79,704.61
Shared room,4.60,724.11


In [38]:
final_df.to_csv(
    "Airbnb_Amsterdam_Listings2.csv",
    index=True,
)